In [35]:
!pip install transformers datasets


In [3]:
import pandas as pd
from datasets import Dataset

# Step 1: Load the data
df = pd.read_csv("/content/drive/My Drive/Kifiya/week 5/final_tokens_labels.csv")

# Display the first few rows to understand the structure
print(df.head())

# Step 2: Process the 'Token' and 'Label' columns
df['Token'] = df['Token'].apply(lambda x: x.split())  # Split tokens
df['Label'] = df['Label'].apply(lambda x: x.split())  # Split labels

# Step 3: Create a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Check the dataset structure
print(dataset)




     Token      Label
0  ባለብዙጥቅም  I-PRODUCT
1   የመስታወት  I-PRODUCT
2     ጎድጓዳ  I-PRODUCT
3     ሳህኖች  I-PRODUCT
4        ከ  I-PRODUCT
Dataset({
    features: ['Token', 'Label'],
    num_rows: 8075
})


In [5]:
import pandas as pd
from datasets import Dataset

# Step 1: Load the data
df = pd.read_csv("/content/drive/My Drive/Kifiya/week 5/final_tokens_labels.csv")

# Display the first few rows to understand the structure
print("DataFrame head:")
print(df.head())

# Step 2: Process the 'Token' and 'Label' columns
df['Token'] = df['Token'].apply(lambda x: x.split())  # Split tokens
df['Label'] = df['Label'].apply(lambda x: x.split())  # Split labels

# Check for empty or missing values
print("\nChecking for missing values:")
print(df.isnull().sum())

# Step 3: Create a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Check the dataset structure
print("\nDataset structure:")
print(dataset)

# Display the first few entries in the dataset to confirm
print("\nDataset samples:")
print(dataset[:5])  # Show first 5 entries


DataFrame head:
     Token      Label
0  ባለብዙጥቅም  I-PRODUCT
1   የመስታወት  I-PRODUCT
2     ጎድጓዳ  I-PRODUCT
3     ሳህኖች  I-PRODUCT
4        ከ  I-PRODUCT

Checking for missing values:
Token    0
Label    0
dtype: int64

Dataset structure:
Dataset({
    features: ['Token', 'Label'],
    num_rows: 8075
})

Dataset samples:
{'Token': [['ባለብዙጥቅም'], ['የመስታወት'], ['ጎድጓዳ'], ['ሳህኖች'], ['ከ']], 'Label': [['I-PRODUCT'], ['I-PRODUCT'], ['I-PRODUCT'], ['I-PRODUCT'], ['I-PRODUCT']]}


In [45]:

# # Step 4: Define the tokenization function
# from transformers import AutoTokenizer

# # Load the tokenizer
# tokenizer = AutoTokenizer.from_pretrained("Davlan/afro-xlmr-base")

# def tokenize_and_align_labels(examples):
#     tokenized_inputs = tokenizer(examples['Token'], truncation=True, is_split_into_words=True)

#     labels = []
#     for i, label_list in enumerate(examples['Label']):
#         word_ids = tokenized_inputs.word_ids(batch_index=i)  # Get the word IDs for the tokens
#         label_ids = []
#         previous_word_idx = None

#         # Aligning labels with tokens
#         for word_idx in word_ids:
#             if word_idx is None:
#                 label_ids.append(-100)  # Special tokens
#             elif word_idx != previous_word_idx:
#                 label_ids.append(label2id[label_list[word_idx]])  # Get label ID for the word
#             else:
#                 label_ids.append(-100)  # Subword tokens
#             previous_word_idx = word_idx

#         labels.append(label_ids)

#     tokenized_inputs["labels"] = labels
#     return tokenized_inputs

# # Step 5: Map the tokenization function to the dataset
# tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

# # Display the tokenized dataset
# print(tokenized_dataset)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/8075 [00:00<?, ? examples/s]

Dataset({
    features: ['Token', 'Label', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 8075
})


In [ ]:
# from transformers import Trainer, TrainingArguments, DataCollatorForTokenClassification

# # Step 1: Set up the training arguments
# training_args = TrainingArguments(
#     output_dir="./results",                     # Output directory
#     evaluation_strategy="epoch",                 # Evaluation strategy to adopt during training
#     learning_rate=2e-5,                          # Learning rate
#     per_device_train_batch_size=16,              # Batch size for training
#     per_device_eval_batch_size=16,               # Batch size for evaluation
#     num_train_epochs=3,                          # Total number of training epochs
#     weight_decay=0.01,                           # Strength of weight decay
#     logging_dir='./logs',                        # Directory for storing logs
#     logging_steps=10,
# )

# # Step 2: Create a data collator
# data_collator = DataCollatorForTokenClassification(tokenizer)

# # Step 3: Initialize the Trainer
# trainer = Trainer(
#     model=model,                                 # The instantiated 🤗 Transformers model to be trained
#     args=training_args,                          # Training arguments, defined above
#     train_dataset=tokenized_dataset,             # The training dataset
#     eval_dataset=None,                           # Optionally, specify an evaluation dataset
#     data_collator=data_collator,                 # Data collator
# )

# # Step 4: Train the model
# trainer.train()




In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import Trainer, TrainingArguments, DataCollatorForTokenClassification
from transformers import AutoTokenizer, AutoModelForTokenClassification

# Step 1: Load the data
df = pd.read_csv("/content/drive/My Drive/Kifiya/week 5/final_tokens_labels.csv")

# Display the first few rows to understand the structure
print("DataFrame head:")
print(df.head())

# Step 2: Process the 'Token' and 'Label' columns
df['Token'] = df['Token'].apply(lambda x: x.split())  # Split tokens
df['Label'] = df['Label'].apply(lambda x: x.split())  # Split labels

# Check for empty or missing values
print("\nChecking for missing values:")
print(df.isnull().sum())

# Step 3: Create a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Step 4: Split the dataset into training and validation sets
dataset = dataset.train_test_split(test_size=0.2)  # Split into training and validation

# Load the model and tokenizer
model_name = "Davlan/afro-xlmr-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define your label to ID mapping
label2id = {
    "B-PRODUCT": 0,
    "I-PRODUCT": 1,
    "B-PRICE": 2,
    "I-PRICE": 3,
    "B-LOC": 4,
    "I-LOC": 5,
    "O": 6,  # 'O' for outside any named entity
}

id2label = {v: k for k, v in label2id.items()}  # Create reverse mapping

# Load the model with the number of labels defined
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(label2id))

# Step 5: Define the tokenization function
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples['Token'], truncation=True, is_split_into_words=True)

    labels = []
    for i, label_list in enumerate(examples['Label']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Get the word IDs for the tokens
        label_ids = []
        previous_word_idx = None

        # Aligning labels with tokens
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Special tokens
            elif word_idx != previous_word_idx:
                label_ids.append(label2id[label_list[word_idx]])  # Get label ID for the word
            else:
                label_ids.append(-100)  # Subword tokens
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Step 6: Map the tokenization function to the dataset
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

# Step 7: Set up the training arguments
training_args = TrainingArguments(
    output_dir="./results",                     # Output directory
    evaluation_strategy="epoch",                 # Updated evaluation strategy
    learning_rate=2e-5,                          # Learning rate
    per_device_train_batch_size=16,              # Batch size for training
    per_device_eval_batch_size=16,               # Batch size for evaluation
    num_train_epochs=3,                          # Total number of training epochs
    weight_decay=0.01,                           # Strength of weight decay
    logging_dir='./logs',                        # Directory for storing logs
    logging_steps=10,
)

# Step 8: Create a data collator
data_collator = DataCollatorForTokenClassification(tokenizer)

# Step 9: Initialize the Trainer
trainer = Trainer(
    model=model,                                 # The instantiated 🤗 Transformers model to be trained
    args=training_args,                          # Training arguments, defined above
    train_dataset=tokenized_dataset['train'],    # Specify the training dataset
    eval_dataset=tokenized_dataset['test'],      # Specify the evaluation dataset
    data_collator=data_collator,                 # Data collator
)

# Step 10: Train the model
trainer.train()

# Step 11: Evaluate the model
eval_results = trainer.evaluate()

# Step 12: Save the model
trainer.save_model("./fine-tuned-model")  # Save the model to the specified directory

# Print evaluation results
print("Evaluation results:", eval_results)


DataFrame head:
     Token      Label
0  ባለብዙጥቅም  I-PRODUCT
1   የመስታወት  I-PRODUCT
2     ጎድጓዳ  I-PRODUCT
3     ሳህኖች  I-PRODUCT
4        ከ  I-PRODUCT

Checking for missing values:
Token    0
Label    0
dtype: int64


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at Davlan/afro-xlmr-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/6460 [00:00<?, ? examples/s]

Map:   0%|          | 0/1615 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.159600,0.251704
2,0.094800,0.175507


In [ ]:
# # Step 5: Evaluate the model (optional)
# # If you have a validation dataset, you can evaluate it:
# # results = trainer.evaluate()

# # Step 6: Save the model
# model.save_pretrained("./fine-tuned-model")
# tokenizer.save_pretrained("./fine-tuned-model")

In [ ]:
!pip install shap lime

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification

# Load your model and tokenizer
model_name = "Davlan/afro-xlmr-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Set model to evaluation mode
model.eval()


In [ ]:
from lime.lime_text import LimeTextExplainer

# Define the function to make predictions
def predict_fn(texts):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=2)
    return predictions.numpy()

# Initialize LIME
explainer = LimeTextExplainer(class_names=label2id.keys())

# Example text for explanation
example_text = "Your example text goes here."

# Generate LIME explanation
exp = explainer.explain_instance(example_text, predict_fn, num_features=10)

# Show the explanation
exp.show_in_notebook(text=example_text)


In [ ]:
import shap

# Function to create SHAP explainer
def create_shap_explainer(model, tokenizer):
    def f(x):
        inputs = tokenizer(x.tolist(), return_tensors="pt", padding=True, truncation=True, max_length=128)
        with torch.no_grad():
            outputs = model(**inputs)
        return torch.softmax(outputs.logits, dim=-1).numpy()

    return shap.KernelExplainer(f, shap.sample(inputs, 100))

# Generate SHAP values for your dataset
explainer = create_shap_explainer(model, tokenizer)
shap_values = explainer.shap_values(example_text)

# Plot SHAP values
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values[0], example_text)


from google.colab import drive
drive.mount('/content/drive')